In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
convlayer = torch.nn.Conv1d(1,256,5)

In [3]:
x = torch.randint(1,10,(1,20)).type(torch.float)

In [4]:
print(x)

tensor([[3., 7., 2., 2., 6., 7., 9., 2., 2., 5., 7., 2., 8., 3., 9., 9., 4., 8.,
         9., 3.]])


In [5]:
print(x.unsqueeze(0).shape)

torch.Size([1, 1, 20])


In [7]:
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):

        super(CausalConv1d, self).__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=0,
            dilation=dilation,
            groups=groups,
            bias=bias)
        
        self.__padding = (kernel_size - 1) * dilation
        
    def forward(self, input):
        return super(CausalConv1d, self).forward(F.pad(input, (self.__padding, 0)))

In [8]:
kernel_size = 3
in_channels=1
out_channels=1
dilation = 1

In [9]:
convlayer = CausalConv1d(in_channels,out_channels,kernel_size,dilation=dilation)

In [10]:
print(convlayer(x.unsqueeze(0)).shape)
print(convlayer(x.unsqueeze(0)))

torch.Size([1, 1, 20])
tensor([[[2.0830, 3.4213, 0.3769, 2.4120, 3.5373, 3.0529, 4.6232, 0.6481,
          2.7996, 3.0137, 3.3049, 0.7645, 5.5534, 0.4547, 6.0188, 3.5379,
          2.0828, 5.4370, 3.9837, 1.3655]]], grad_fn=<SqueezeBackward1>)


In [11]:
__padding = (kernel_size - 1) * dilation
x_pad = F.pad(x, (__padding, 0))

In [12]:
torch.dot(convlayer.weight[0][0],x_pad[0][0:3])+convlayer.bias[0]

tensor(2.0830, grad_fn=<AddBackward0>)